In [2]:
import pandas as pd
import numpy as np

In [775]:
df = pd.read_csv("raw/written_cleaned1804.csv")

In [776]:
def setTokenLength(groupByDoc):
    groupByDoc['token_length'] = groupByDoc.token_id.max()
    return groupByDoc

df = df.groupby('doc_id').apply(setTokenLength)

In [777]:
# calculate dependency_length for each token
for index, row in df.iterrows():
    if row["head_token_id"] == 0:
        df.at[index,'dependency_length'] = 0
    else:
        df.at[index,'dependency_length'] = df.at[index,'token_id'] - df.at[index,'head_token_id']

In [778]:
# calculate jumlah dependency_length for each sentence
current_id = df.loc[0,'doc_id']
sum_dependency_length = 0
df["sum_dependency_length"] = np.NaN

for index, row in df.iterrows():
    if row["doc_id"] == current_id:
        sum_dependency_length += abs(df.at[index,"dependency_length"])
        df.at[index, "sum_dependency_length"] = sum_dependency_length
    else:
        sum_dependency_length = abs(df.at[index,"dependency_length"])
        current_id = row["doc_id"]

In [779]:
#Find Mean Dependency Distance
df["mdd"] = df.sum_dependency_length / (df.token_length - 1)

In [780]:
def calculateMddPosAndNeg(group):
    group["mdd_positive"] = np.mean(group[group.dependency_length > 0].dependency_length)
    group["mdd_negative"] = np.abs(np.mean(group[group.dependency_length < 0].dependency_length))
    return group

df = df.groupby('doc_id').apply(calculateMddPosAndNeg)

In [781]:
def calculateSumDepPosAndNeg(group):
    group["sum_positive"] = np.sum(group[group.dependency_length > 0].dependency_length)
    group["sum_negative"] = np.abs(np.sum(group[group.dependency_length < 0].dependency_length))
    return group

df = df.groupby('doc_id').apply(calculateSumDepPosAndNeg)

In [15]:
df.loc[:,["doc_id","token_length","token_id", "head_token_id","dependency_length","sum_dependency_length","mdd","sum_positive", "sum_negative","mdd_positive", "mdd_negative"]]


,doc_id,token_length,token_id,head_token_id,dependency_length,sum_dependency_length,mdd,sum_positive,sum_negative,mdd_positive,mdd_negative
0,doc1,5,1,3,-2.0,2.0,0.500000,3.0,3.0,1.500000,1.500000
1,doc1,5,2,3,-1.0,3.0,0.750000,3.0,3.0,1.500000,1.500000
2,doc1,5,3,0,0.0,3.0,0.750000,3.0,3.0,1.500000,1.500000
3,doc1,5,4,3,1.0,4.0,1.000000,3.0,3.0,1.500000,1.500000
4,doc1,5,5,3,2.0,6.0,1.500000,3.0,3.0,1.500000,1.500000
5,doc2,12,1,4,-3.0,NaN,NaN,13.0,6.0,1.857143,1.500000
6,doc2,12,2,1,1.0,4.0,0.363636,13.0,6.0,1.857143,1.500000
7,doc2,12,3,4,-1.0,5.0,0.454545,13.0,6.0,1.857143,1.500000
8,doc2,12,4,0,0.0,5.0,0.454545,13.0,6.0,1.857143,1.500000
9,doc2,12,5,4,1.0,6.0,0.545455,13.0,6.0,1.857143,1.500000


In [31]:
df = pd.read_csv("out/audio_cleaned1804_out.csv")

In [20]:
#create a new column new_token_id filled with a randomized order of the token_id

import random

def shuffleTokenId(groupByDoc):
    groupByDoc['new_token_id'] = random.sample(range(1,len(groupByDoc)+1), len(groupByDoc))
    return groupByDoc

In [21]:
def shuffleAndAppend(df, reps):
    appendedDf = pd.DataFrame()
    for i in range(reps):
        df['shuffle_id'] = df['doc_id'] + "-" + str(i)
        appendedDf = appendedDf.append(df.groupby('doc_id').apply(shuffleTokenId))
    return appendedDf

In [22]:
# create a new column prev_rnd_head_token_id filled with a lookup value of the previous head_token_id from rnd_pos_id
def setPrev_head_token_id(group):
    for i, irow in group.iterrows():
        for j, jrow in group.iterrows():
            if irow["new_token_id"] == jrow["token_id"]:
                group.at[i, "prev_head_token_id"] = jrow["head_token_id"]
                break  
    return group

In [23]:
def setNew_head_token_id(group):
    for i, irow in group.iterrows():
        for j, jrow in group.iterrows():
            if irow["prev_head_token_id"] == jrow["new_token_id"]:
                group.at[i, "new_head_token_id"] = jrow["token_id"]
                break
            else:
                group.at[i, "new_head_token_id"] = 0
    return group

In [24]:
# calculate new dependency_length after shuffling
def calculateNewDependencyLength(df):
    for index, row in df.iterrows():
        if row["new_head_token_id"] == 0:
            df.at[index,'new_dependency_length'] = 0
        else:
            df.at[index,'new_dependency_length'] = df.at[index,'token_id'] - df.at[index,'new_head_token_id']
    return df

In [25]:
# calculate jumlah dependency_lenght for each sentence
def calculateNewSumDependencyLength(group):
    group['new_sum_dependency_length'] = np.sum(abs(group['new_dependency_length']))
    return group

In [26]:
#Find Mean Dependency Distance
def calculateNewMdd():
    df["new_mdd"] = df.new_sum_dependency_length / (df.token_length - 1)
    return df

In [27]:
def calculateNewAvgPosAndNegDependency(group):
    group["new_avg_positive_dependency"] = np.mean(group[group.new_dependency_length > 0].new_dependency_length)
    group["new_avg_negative_dependency"] = np.abs(np.mean(group[group.new_dependency_length < 0].new_dependency_length))
    return group

In [28]:
def shuffleAndCalculate(df):
    df = df.groupby('doc_id').apply(shuffleTokenId) 
    df = df.groupby('doc_id').apply(setPrev_head_token_id)
    df = df.groupby('doc_id').apply(setNew_head_token_id)
    df = calculateNewDependencyLength(df)
    df = df.groupby('doc_id').apply(calculateNewSumDependencyLength)
    df = df.groupby('doc_id').apply(calculateNewAvgPosAndNegDependency)
    return df

In [29]:
def repeatShuffle(df, reps):
    df_new = pd.DataFrame()
    for i in range(reps):
        temp = shuffleAndCalculate(df)
        temp["shuffle_id"] = str(i)
        temp.doc_id = temp.doc_id + "-s" + temp.shuffle_id 
        df_new = df_new.append(temp)
    
    return df_new

In [ ]:
#85 docs
#repeatShuffle(df, 100).to_csv("allshuffle100_out.csv")

In [18]:
df.loc[:,["doc_id","token_length","token_id", "head_token_id","dependency_length","sum_dependency_length","new_token_id","prev_head_token_id","new_head_token_id", "new_dependency_length", "new_sum_dependency_length"]]


/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,doc_id,token_length,token_id,head_token_id,dependency_length,sum_dependency_length,new_token_id,prev_head_token_id,new_head_token_id,new_dependency_length,new_sum_dependency_length
0,doc1,5,1,3,-2.0,2.0,NaN,NaN,NaN,NaN,NaN
1,doc1,5,2,3,-1.0,3.0,NaN,NaN,NaN,NaN,NaN
2,doc1,5,3,0,0.0,3.0,NaN,NaN,NaN,NaN,NaN
3,doc1,5,4,3,1.0,4.0,NaN,NaN,NaN,NaN,NaN
4,doc1,5,5,3,2.0,6.0,NaN,NaN,NaN,NaN,NaN
5,doc2,12,1,4,-3.0,NaN,NaN,NaN,NaN,NaN,NaN
6,doc2,12,2,1,1.0,4.0,NaN,NaN,NaN,NaN,NaN
7,doc2,12,3,4,-1.0,5.0,NaN,NaN,NaN,NaN,NaN
8,doc2,12,4,0,0.0,5.0,NaN,NaN,NaN,NaN,NaN
9,doc2,12,5,4,1.0,6.0,NaN,NaN,NaN,NaN,NaN


In [32]:
df5 = df[df.token_length == 5]
df12 = df[df.token_length == 12]
df22 = df[df.token_length == 22]
df_sampling = df5.append(df12)
df_sampling = df_sampling.append(df22)

In [ ]:
df_out = repeatShuffle(df_sampling, 100).to_csv("out/5-12-22shuffle100_out.csv")